# WORK In PROGRESS Convert EHR data to format used in this repo

This repository comes with a [sample dataset](../data-synthetic/ed_visits.csv) of ED visits, which are saved in a particular format. You'll see that each row has a snapshot_date and prediction_time. When combined, these identify a moment in time, during a patient's visit to the Emergency Department, at which we accessed everything known about them up to that point. In the row, information about the visit appears in summary form. For example, there are columns for visited_majors and visited_resus, as well as a current_location_type. One visit may have multiple rows, and there data about a given visit is repeated. 

This structure is very different from a Electronic Health Record data warehouse or Patient Adminisration System, which would usually have relational structure where information is not repeated. 

However, this structure is necessary for developing a predictive model of admission. To train our models, we must mimic the situation that would appear when making a prediction about a real visit that hasn't ended yet. We therefore need our dataset to look as if visits are unfinished. 

This notebooks suggests how you might turn our own EHR or PAS data into the structure required. 

In [1]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

## DB connection script

In [2]:
# An example of how to set up database connection (NOT RUN)
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

def get_credentials():
    with open('../../secret-emap', 'r') as file:
        username = file.readline().strip()
        password = file.readline().strip()
        database_host = file.readline().strip()
        database_name = file.readline().strip()
        database_port = file.readline().strip()
        return username, password, database_host, database_name, database_port

# Get credentials from secret file
username, password, database_host, database_name, database_port = get_credentials()

# Database connection URL
DATABASE_URL = f"postgresql://{username}:{password}@{database_host}:{database_port}/{database_name}"

# Create engine
engine = create_engine(DATABASE_URL)

## Retrieve the data

In the cell below I'm doing some housekeeping to tell the notebook where to find the config file and loading some functions that will mask the real data (by hashing the csn visit number, and shifting dates into the future) so that no patient identifiers are revealed

In [28]:
from patientflow.mask import hash_csn, shift_dates
from patientflow.load import set_project_root
project_root = set_project_root()

from patientflow.load import set_file_paths
from patientflow.load import load_config_file
from datetime import timedelta


# set file paths
data_folder_name = 'data-public'
data_file_path = project_root / data_folder_name

data_file_path, _, _, config_path = set_file_paths(project_root, 
               data_folder_name=data_folder_name)

# load params
params = load_config_file(project_root / 'config-uclh.yaml')

arrived_after = params["start_validation_set"].isoformat()
arrived_before = (params["start_validation_set"] + timedelta(days = 2)).isoformat()


Inferred project root: /home/jovyan/work/zella/patientflow
Configuration will be loaded from: /home/jovyan/work/zella/patientflow/config.yaml
Data files will be loaded from: /home/jovyan/work/zella/patientflow/data-public
Trained models will be saved to: /home/jovyan/work/zella/patientflow/trained-models/public
Images will be saved to: /home/jovyan/work/zella/patientflow/trained-models/public/media


Here I'm loading a very simple dataset that will tell me the

- hospital visit start and end times
- start and end times at each location

In [22]:

# retreive salt to mask file dates
with open('../../seed', 'r') as file:
    seed = file.readline().strip()

# get the data
from pathlib import Path
import pandas as pd

# create parameters dictionary
sql_params = {
    'arrived_after': arrived_after,
    'arrived_before': arrived_before
}

# set up SQL query
SQL_DIR = Path("/home/jovyan/work/zella/zbeds/sql")
subquery = (SQL_DIR / "EMAP_ed_subquery.sql").read_text()
mainquery = (SQL_DIR / "EMAP_test_script.sql").read_text()
final_query = mainquery.replace('[subquery]', f'({subquery})')

# execute the combined query
df = pd.read_sql(
    final_query,
    engine,
    params=sql_params
)

# Hash the csns before displaying the df
df = hash_csn(df, seed)

# shift the dates before displaying the df
df = shift_dates(df, seed)

The output below also shows the location string where the patient was, and the arrival method

## Identify moment of departure from ED

Using the data above I'm going to work out when each patient left the Emergency Department. Any location string that starts with 'ED' or '1020100166' or '1020100170' is an ED location in these data. I'll find the last ED location for each vist, and the timestamp at which the patient left that location. This will be marked as their ed_departure

In [23]:
# sort the data 
df.sort_values(['csn', 'location_arrival'], inplace= True)

# create a mask for ED locations using string operations
ed_mask = (
    df['location_string'].str.startswith('ED^') |
    df['location_string'].str.startswith('1020100166^') |
    df['location_string'].str.startswith('1020100170^')
)

# Filter for ED locations and group by csn to find first departure
ed_departure = (
    df[ed_mask]
    .groupby('csn')['location_departure']
    .max()
    .reset_index()
    .rename(columns={'location_departure': 'ed_departure'})
)

# merge this back with original dataframe:
df_with_departure = df.merge(
    ed_departure,
    on='csn',
    how='left'
)

## Identify whether patient was admitted

Above I worked out the timestamp of patient's leaving the ED, and labelled it ed_departure. After this point, they either left the hospital, or went to a non-ED location. Here I'm going to deem that they were admitted if they have another location after ed_departure

In [24]:
# Calculate admission status
def determine_admission(group):
    # Get rows after first ED departure
    post_ed = group[group['location_arrival'] >= group['ed_departure']]
    return len(post_ed) > 0

# Group by csn and apply the admission check
admissions = (
    df_with_departure
    .groupby('csn')
    .apply(determine_admission)
    .reset_index()
    .rename(columns={0: 'is_admitted'})
)

# Merge admission status back to dataframe
df_with_admission_status = df_with_departure.merge(
    admissions,
    on='csn',
    how='left'
)

In [25]:
df_with_admission_status.head(20)

,csn,hospital_arrival,hospital_departure,arrival_method,location_arrival,location_departure,location_string,ed_departure,is_admitted
0,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-07 19:48:13,2035-09-07 19:49:00,ED^null^null,2035-09-07 22:21:00,True
1,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-07 19:49:00,2035-09-07 22:13:00,ED^UCHED PAEDS TRIAGE^PAED WR,2035-09-07 22:21:00,True
2,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-07 22:13:00,2035-09-07 22:21:00,ED^UCHED OTF POOL^OTF,2035-09-07 22:21:00,True
3,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-07 22:21:00,2035-09-08 07:45:00,T11S^T11S BY05*^BY05-27,2035-09-07 22:21:00,True
4,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-08 07:45:00,2035-09-08 09:49:00,THP3^UCHP03THR POOL01^THR,2035-09-07 22:21:00,True
5,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-08 09:49:00,2035-09-08 21:01:00,T11S^T11S BY05*^BY05-27,2035-09-07 22:21:00,True
6,00eaa24aa2f3,2035-09-07 19:49:00,2035-09-09 11:45:00,Walk-in,2035-09-08 21:01:00,2035-09-09 11:45:00,T11S^T11S WAITING^WAIT,2035-09-07 22:21:00,True
7,01460d38da77,2035-09-07 12:27:00,2035-09-14 14:41:00,Walk-in,2035-09-07 12:18:25,2035-09-07 12:18:26,ED^null^null,2035-09-07 22:16:00,True
8,01460d38da77,2035-09-07 12:27:00,2035-09-14 14:41:00,Walk-in,2035-09-07 12:18:26,2035-09-07 12:27:00,ED^null^null,2035-09-07 22:16:00,True
9,01460d38da77,2035-09-07 12:27:00,2035-09-14 14:41:00,Walk-in,2035-09-07 12:27:00,2035-09-07 12:51:00,ED^UCHED ADULT TRIAGE^NONE,2035-09-07 22:16:00,True


In [ ]:
# Modify final_df to include only ED rows

# df_final = df_with_admission_status[df_with_admission_status.location_departure <= df_with_admission_status.ed_departure]

## Set up an array of snapshot datetimes

Above we loaded the parameters file, which says what times we want to run our predictions. These are saved as (hour, minute) tuples, as shown here.

In [30]:
snapshot_times = params["prediction_times"]
print(snapshot_times)

[(6, 0), (9, 30), (12, 0), (15, 30), (22, 0)]


The function below will shift the snapshot_dates into the future, to align with the shifted data

In [32]:
snapshot_dates = shift_dates(
    data=None,
    seed=seed,
    start_date=arrived_after,
    end_date=arrived_before
)
snapshot_dates

[datetime.date(2035, 9, 6), datetime.date(2035, 9, 7)]

## Create snapshots dataset

Combining the snapshot dates and times provides a series of prediction_times at which we sample the data on the patients in ED. At each of those times, we want to know which patients were in the ED. We will save their arrival method, since this is known at the beginning of the visit 


In [44]:
from datetime import datetime, time
import pandas as pd

def create_snapshots(df, snapshot_times, snapshot_dates):
    # Create empty list to store all results
    all_results = []
    
    # For each combination of date and time
    for date in snapshot_dates:
        for hour, minute in snapshot_times:
            snapshot_datetime = datetime.combine(
                date, 
                time(hour=hour, minute=minute)
            )
            
            # Filter dataframe for this snapshot
            mask = (df['hospital_arrival'] <= snapshot_datetime) & (df['ed_departure'] > snapshot_datetime) 
            snapshot_df = df[mask].copy()  # Create copy to avoid SettingWithCopyWarning
            
            # Add snapshot information columns
            snapshot_df['snapshot_date'] = date
            snapshot_df['snapshot_time'] = [(hour, minute)] * len(snapshot_df)
            
            # Append to results list
            all_results.append(snapshot_df)
    
    # Combine all results into single dataframe
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        snapshot_cols = ['snapshot_date', 'snapshot_time']
        other_cols = ['csn', 'location_string']#[col for col in final_df.columns if col not in snapshot_cols]
        final_df = final_df[snapshot_cols + other_cols]
    else:
        # Create empty dataframe with correct columns if no results found
        final_df = pd.DataFrame(columns=list(df.columns) + ['snapshot_date', 'snapshot_time', 'snapshot_datetime'])
    
    return final_df
snapshots_df = create_snapshots(df_with_admission_status, snapshot_times, snapshot_dates)

From the output below, we can multiple rows per visit and snapshot. This is because a patient can visit more than one location during their stay. What is shown below does not make much sense, as we haven't taken into account whether the locations were visited before or after the snapshot. 

In [46]:
snapshots_df.head(10)

,snapshot_date,snapshot_time,csn,location_string
0,2035-09-06,"(6, 0)",02dd027d9c4a,ED^null^null
1,2035-09-06,"(6, 0)",02dd027d9c4a,ED^UCHED UTC POOL02^UTC TZ
2,2035-09-06,"(6, 0)",1115ec4384c3,ED^null^null
3,2035-09-06,"(6, 0)",1115ec4384c3,ED^UCHED RAT CHAIR^RAT-CHAIR
4,2035-09-06,"(6, 0)",1115ec4384c3,ED^UCHED RAT04^RAT-04
5,2035-09-06,"(6, 0)",1115ec4384c3,1020100166^SDEC 18^18 SDEC
6,2035-09-06,"(6, 0)",11db227f9cc4,ED^null^null
7,2035-09-06,"(6, 0)",38b00f60cfd4,ED^null^null
8,2035-09-06,"(6, 0)",38b00f60cfd4,ED^UCHED UTC POOL02^UTC TZ
9,2035-09-06,"(6, 0)",41b23149529b,ED^null^null


So the code below is the same, but this time we'll add a function that will summarise of the location data, to ensure that we only have one row for each combination of snapshot and csn

WORK IN PROGRESS